In [1]:
from bancor_research import DEFAULT, DataFrame
from bancor_research.bancor_simulator.v3.spec.network import BancorDapp as sBancorDapp
from bancor_research.bancor_emulator.v3.spec.network import BancorDapp as eBancorDapp

# Bootstrapping Behavior

To illustrate how the system works, we will constrain the system to an oversimplified case with only three pools: ETH, wBTC and LINK, in addition to the BNT omnipool. Assume that these pools have just recently earned whitelist status, they each have a BNT funding limit of 40,000 BNT, and all are completely empty.

In [2]:
whitelisted_tokens = {tkn_name : DEFAULT.WHITELIST[tkn_name] for tkn_name in DEFAULT.WHITELIST if tkn_name in ['eth', 'link', 'wbtc']}

In [3]:
bnt_funding_limit = '40000'

The protocol is in the 'bootstrapping phase' for all three assets. UserState can still interact with the system and deposit liquidity - which is essential for the process. The system needs to attract sufficient quantities of each token to overcome the minimum liquidity threshold prior to the activation of each liquidity pool, respectively. The default minimum liquidity threshold is 10,000 BNT (previously 1,000 BNT in an earlier draft of this proposal), meaning that the pool must mint 10,000 BNT during its activation, and therefore there must be at least a commensurate value of TKN available in the vault.

In [4]:
bnt_min_liquidity = '10000'

Assume that the prices of each asset are as follows: BNT = 2.50, LINK = 15.00, ETH = 2,500.00, wBTC = 40,000.00. Therefore, to bootstrap each pool, at least $25,000 of each asset must be available in the vault.

In [5]:
# Recall from an earlier chapter that we already defined the cooldown period and the exit fees as follows.
cooldown_time = DEFAULT.COOLDOWN_TIME
withdrawal_fee = DEFAULT.WITHDRAWAL_FEE
network_fee = DEFAULT.NETWORK_FEE

iter_limit = 10000

price_feeds = DataFrame({
    'INDX':[0 for i in range(iter_limit + 1)],
    'bnt':[2.5 for i in range(iter_limit + 1)],
    'link':[15.00 for i in range(iter_limit + 1)],
    'eth':[2500.00 for i in range(iter_limit + 1)],
    'wbtc':[40000.00 for i in range(iter_limit + 1)]
})

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
dapps = [
    BancorDapp(
        whitelisted_tokens=whitelisted_tokens,
        network_fee=network_fee,
        cooldown_time=cooldown_time,
        withdrawal_fee=withdrawal_fee,
        bnt_min_liquidity=bnt_min_liquidity,
        price_feeds=price_feeds
    )
    for BancorDapp in [sBancorDapp, eBancorDapp]
]

def assertAndDisplay(decimals = -1):
    frames = [v3.describe(decimals) for v3 in dapps]
    diff = frames[0].compare(frames[1])
    assert diff.empty, diff
    return frames[0]


During the bootstrap phase, Alice deposits 100 ETH, Bob deposits 100 bnwBTC, and Charlie deposits 10,000 LINK. By default, the initial pool token value is 1:1 with the underlying asset, and only changes after revenue begins accumulating. Therefore, each of these characters receive 100 bnETH, 100 bnwBTC and 1,000 bnLINK pool tokens for their contribution, respectively,

In [6]:
for v3 in dapps:
    v3.create_user('Alice')
    v3.create_user('Bob')
    v3.create_user('Charlie')

    v3.set_user_balance(tkn_amt='101', tkn_name='eth', user_name='Alice')
    v3.set_user_balance(tkn_amt='101', tkn_name='wbtc', user_name='Bob')
    v3.set_user_balance(tkn_amt='10001', tkn_name='link', user_name='Charlie')

However, note that the trading liquidity available on the pools remains unchanged until the BancorDAO msig signers initialize each pool.

The condition to initialize a pool is that there must be at least 10,000 BNT ($25,000 at the quoted price) worth of each token - in this case all three satisfy the criteria. The BancorDAO msig signers then sign a transaction with the rate of BNT/TKN for each of the bootstrapping assets. In this case, BNT/ETH = 1,000, BNT/wBTC = 16,000, BNT/LINK =  6. The trading liquidity on each pool begins with exactly 10,000 BNT and the commensurate amount of TKN as determined by the rates supplied by the msig signers.

It is important to note here that each of these pools have been stated to have a 100,000 BNT funding limit, 10× higher than the bootstrapping depth. This is because the pools should equilibrate with the rest of the market and grow in a controlled manner. Therefore, only a fraction of the available ETH, wBTC and LINK has been committed to trading at this stage. The other important change is that the BNT that was created by the protocol causes its own pool token to be issued, bnBNT,

Fortunately, the rate at which the pool can grow is independent of the new liquidity additions, and is determined entirely by the token balances available in the vault. To demonstrate, suppose that each user deposits just 1 more TKN each to their respective pools; Alice deposits 1 ETH, Bob deposits 1 wBTC, and Charlie deposits 1 LINK. Each of these tokens is accepted directly to the vault. Assume that no trading has yet occurred on the pool, therefore the valuation of each pool token remains at 1:1 for the time being. Therefore, each user is issued one additional bnTKN.

In [7]:
for v3 in dapps:
    v3.deposit(tkn_amt='100', tkn_name='eth', user_name='Alice')
    v3.deposit(tkn_amt='10000', tkn_name='link', user_name='Charlie')
    v3.deposit(tkn_amt='100', tkn_name='wbtc', user_name='Bob')

assertAndDisplay(decimals=6)

bnt         eth          link  \
1 Account Alice                     0.000000    1.000000      0.000000   
          Bob                       0.000000    0.000000      0.000000   
          Charlie                   0.000000    0.000000      1.000000   
2 Pool    a: TKN Staked Balance            0  100.000000  10000.000000   
          b: TKN Trading Liquidity         0    0.000000      0.000000   
          c: BNT Trading Liquidity         0    0.000000      0.000000   
          d: BNT Current Funding           0    0.000000      0.000000   
          e: Spot Rate                     0    0.000000      0.000000   
          f: Average Rate                  0    0.000000      0.000000   
          g: Average Inverse Rate          0    0.000000      0.000000   
3 Network Master Vault              0.000000  100.000000  10000.000000   
          Protection Vault          0.000000    0.000000      0.000000   
          Protocol Equity                  0           0             0   
          Rewards Vault                    0           0             0   

                                          wbtc     bnbnt       bneth  \
1 Account Alice                       0.000000  0.000000  100.000000   
          Bob                         1.000000  0.000000    0.000000   
          Charlie                     0.000000  0.000000    0.000000   
2 Pool    a: TKN Staked Balance     100.000000         0           0   
          b: TKN Trading Liquidity    0.000000         0           0   
          c: BNT Trading Liquidity    0.000000         0           0   
          d: BNT Current Funding      0.000000         0           0   
          e: Spot Rate                0.000000         0           0   
          f: Average Rate             0.000000         0           0   
          g: Average Inverse Rate     0.000000         0           0   
3 Network Master Vault              100.000000         0           0   
          Protection Vault            0.000000         0           0   
          Protocol Equity                    0  0.000000    0.000000   
          Rewards Vault                      0  0.000000    0.000000   

                                          bnlink      bnwbtc      vbnt  
1 Account Alice                         0.000000    0.000000  0.000000  
          Bob                           0.000000  100.000000  0.000000  
          Charlie                   10000.000000    0.000000  0.000000  
2 Pool    a: TKN Staked Balance                0           0         0  
          b: TKN Trading Liquidity             0           0         0  
          c: BNT Trading Liquidity             0           0         0  
          d: BNT Current Funding               0           0         0  
          e: Spot Rate                         0           0         0  
          f: Average Rate                      0           0         0  
          g: Average Inverse Rate              0           0         0  
3 Network Master Vault                         0           0         0  
          Protection Vault                     0           0         0  
          Protocol Equity               0.000000    0.000000         0  
          Rewards Vault                 0.000000    0.000000         0

In [8]:
for v3 in dapps:
    v3.deposit(tkn_amt='1', tkn_name='eth', user_name='Alice')
    v3.deposit(tkn_amt='1', tkn_name='link', user_name='Charlie')
    v3.deposit(tkn_amt='1', tkn_name='wbtc', user_name='Bob')

    for tkn_name in whitelisted_tokens:
        v3.enable_trading(tkn_name)

assertAndDisplay(decimals=6)

bnt           eth          link  \
1 Account Alice                         0.000000      0.000000      0.000000   
          Bob                           0.000000      0.000000      0.000000   
          Charlie                       0.000000      0.000000      0.000000   
2 Pool    a: TKN Staked Balance                0    101.000000  10001.000000   
          b: TKN Trading Liquidity             0     20.000000   3333.333333   
          c: BNT Trading Liquidity             0  20000.000000  20000.000000   
          d: BNT Current Funding               0  20000.000000  20000.000000   
          e: Spot Rate                         0   1000.000000      6.000000   
          f: Average Rate                      0   1000.000000      6.000000   
          g: Average Inverse Rate              0      0.001000      0.166667   
3 Network Master Vault              60000.000000    101.000000  10001.000000   
          Protection Vault              0.000000      0.000000      0.000000   
          Protocol Equity                      0             0             0   
          Rewards Vault                        0             0             0   

                                            wbtc         bnbnt       bneth  \
1 Account Alice                         0.000000      0.000000  101.000000   
          Bob                           0.000000      0.000000    0.000000   
          Charlie                       0.000000      0.000000    0.000000   
2 Pool    a: TKN Staked Balance       101.000000             0           0   
          b: TKN Trading Liquidity      1.250000             0           0   
          c: BNT Trading Liquidity  20000.000000             0           0   
          d: BNT Current Funding    20000.000000             0           0   
          e: Spot Rate              16000.000000             0           0   
          f: Average Rate           16000.000000             0           0   
          g: Average Inverse Rate       0.000062             0           0   
3 Network Master Vault                101.000000             0           0   
          Protection Vault              0.000000             0           0   
          Protocol Equity                      0  60000.000000    0.000000   
          Rewards Vault                        0      0.000000    0.000000   

                                          bnlink      bnwbtc      vbnt  
1 Account Alice                         0.000000    0.000000  0.000000  
          Bob                           0.000000  101.000000  0.000000  
          Charlie                   10001.000000    0.000000  0.000000  
2 Pool    a: TKN Staked Balance                0           0         0  
          b: TKN Trading Liquidity             0           0         0  
          c: BNT Trading Liquidity             0           0         0  
          d: BNT Current Funding               0           0         0  
          e: Spot Rate                         0           0         0  
          f: Average Rate                      0           0         0  
          g: Average Inverse Rate              0           0         0  
3 Network Master Vault                         0           0         0  
          Protection Vault                     0           0         0  
          Protocol Equity               0.000000    0.000000         0  
          Rewards Vault                 0.000000    0.000000         0

and the balances of the staking ledger are updated accordingly.